#**Import Lib**

In [ ]:
import os
import tensorflow as tf

In [ ]:
os.environ['KAGGLE_USERNAME'] = "phongngochoangnguyen"
os.environ['KAGGLE_KEY'] = "6ea0056aa0eef73e7925731ea93dbb6f"
!kaggle competitions download -c ds3-datathon-traffic-signs

ds3-datathon-traffic-signs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#!unzip ds3-datathon-traffic-signs.zip

In [ ]:
data_dir="/content/traffic_signs/Train"
training_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    batch_size=16,
    image_size=(32,32),
    shuffle=True,
    seed=505,
    validation_split=0.25,
    subset='training',
)
validation_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    batch_size=16,
    image_size=(32,32),
    shuffle=True,
    seed=505,
    validation_split=0.25,
    subset='validation',
)

Found 39209 files belonging to 43 classes.
Using 29407 files for training.
Found 39209 files belonging to 43 classes.
Using 9802 files for validation.


In [ ]:
autotune=tf.data.experimental.AUTOTUNE
training_dataset=training_dataset.prefetch(buffer_size=autotune)
validation_dataset=validation_dataset.prefetch(buffer_size=autotune)

#**Define Layers**

In [ ]:
preprocess=tf.keras.applications.densenet.preprocess_input

In [ ]:
rescaling=tf.keras.layers.experimental.preprocessing.Rescaling(2./255,offset=-1)

In [ ]:
global_average_pooling=tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
prediction_layer=tf.keras.layers.Dense(43,activation='softmax')

In [ ]:
img_shape=(32,32,3)
pretrained_model=tf.keras.applications.densenet.DenseNet201(input_shape=img_shape,include_top=False,weights='imagenet')

In [ ]:
data_augmentation=tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

#**Model**

In [ ]:
inputs=tf.keras.Input(img_shape)
x=data_augmentation(inputs)
x=preprocess(x)
x=pretrained_model(x,training=False)
x=global_average_pooling(x)
x=tf.keras.layers.Dropout(0.2)(x)
x=tf.keras.layers.Dense(128,activation='relu')(x)
x=tf.keras.layers.Dropout(0.2)(x)
outputs=prediction_layer(x)
model=tf.keras.Model(inputs,outputs)

In [ ]:
model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.0001),
    tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(
    min_delta=0.005,
    patience=5,
    restore_best_weights=True,
)

In [ ]:
history=model.fit(
    training_dataset,
    validation_data=validation_dataset,
    epochs=3,
)

Epoch 1/3
1838/1838 [==============================] - 276s 150ms/step - loss: 1.2436 - accuracy: 0.6137 - val_loss: 0.4541 - val_accuracy: 0.8464
Epoch 2/3
1838/1838 [==============================] - 268s 146ms/step - loss: 0.5142 - accuracy: 0.8272 - val_loss: 0.3417 - val_accuracy: 0.8759
Epoch 3/3
1838/1838 [==============================] - 268s 146ms/step - loss: 0.3455 - accuracy: 0.8870 - val_loss: 0.2422 - val_accuracy: 0.9179


#**Test**

In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv('/content/traffic_signs/traffic_test.csv')

In [ ]:
print(data['Path'])

0        Test/00000.png
1        Test/00001.png
2        Test/00002.png
3        Test/00003.png
4        Test/00004.png
              ...      
12625    Test/12625.png
12626    Test/12626.png
12627    Test/12627.png
12628    Test/12628.png
12629    Test/12629.png
Name: Path, Length: 12630, dtype: object


In [ ]:
result=[]
conf=[]
for path in data['Path']:
    image=os.path.join('/content/traffic_signs',path)
    image=tf.keras.utils.load_img(image,target_size=(32,32,3))
    image_arr=tf.keras.utils.array_to_img(image)
    image_bat=tf.expand_dims(image_arr,0)
    predict=model.predict(image_bat)
    result.append(np.argmax(predict))
    conf.append(np.max(predict))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 41ms/step


In [ ]:
id=data['id']

In [ ]:
len(result)

12630

#**Export**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
# create a DataFrame
data = {'id':id,
        'Output': result}
df = pd.DataFrame(data)
print(df)

          id  Output
0          0       8
1          1       1
2          2      32
3          3      32
4          4       3
...      ...     ...
12625  12625       4
12626  12626      27
12627  12627      38
12628  12628      40
12629  12629       2

[12630 rows x 2 columns]


In [ ]:
df.to_csv('/content/drive/My Drive/mydata.csv', index=False)